In [ ]:
"""
<심층 신경망(DNN)>
 - 입력계층과 출력계층 사이에 여러 계층이 포함된 경우
 - 중간에 포함된 계층을 -> 은닉계층(Hidden Layer)이라고 표현함
 - 은닉계층은 입력된 데이터를 다음 계층으로 넘겨주기 위한 활성화 함수가 필요함
   (보통 relu를 주로 사용함)
"""

In [ ]:
"""
1. 패션데이터셋 읽어들이기 : 변수는 기존과 동일
2. 데이터 스케일링 : 변수는 기존과 동일
3. 훈련 : 검증 = 8:2로 분리 : 변수는 기존과 동일
"""

In [1]:
"""
<텐서플로 프레임워크 사용>
 - 사용 라이브러리 : keras
"""

### 텐서플로우 프레임워크(패키지라고도 칭함)
import tensorflow as tf

### 케라스 라이브러리 불러들이기
from tensorflow import keras

### 시각화 라이브러리 불러들이기
import matplotlib.pyplot as plt

### 넘파이
import numpy as np

### 훈련 및 테스트 데이터로 분류하는 라이브러리
from sklearn.model_selection import train_test_split

import os
os.environ["TF_DETERMINISTIC_OPS"] = "0"


### 딥러닝 랜덤 규칙 정의하기
# - 딥러닝에서의 랜덤 규칙은 항상 일정하게 유지되지는 않음
# - 딥러닝 모델 내부에서 훈련을 위한 데이터를 임의로 추출하여 사용(사람이 관여 안함)
tf.keras.utils.set_random_seed(42)
# tf.config.experimental.enable_op_determinism()

In [2]:
### keras에서 제공하는 데이터 라이브러리 : Fashion MNIST
# - 이미지 데이터로 되어있음..

### 데이터셋 읽어들이기
# - 반환되는 데이터 : train(독립, 종속), test(독립, 종속) 총 4개 데이터 변수로 반환됨
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [ ]:
### 이미지 픽셀 데이터를 0~1사이의 값으로 변환하기
# - 이미지 픽셀 데이터의 범위는 0~255의 범위값으로 구성되어 있음
# - 따라서, 각 픽셀의 값을 255로 나눠주면, 0~1사이의 값으로 일반화가 가능함
# - 훈련 및 테스트 독립변수의 데이터를 모두 255로 나누어서 전처리하기
#   (255.0의 소숫점으로 나눠줍니다.)
train_scaled_255 = train_input / 255.0
test_scaled_255  = test_input / 255.0

In [ ]:
### 훈련(train_scaled, train_target)
### 검증(val_scaled, val_target)
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled_255, train_target, test_size=0.2, random_state=42
)

print(train_scaled.shape,   train_target.shape)
print(val_scaled.shape,     val_target.shape)
print(test_scaled_255.shape, test_target.shape)

### 심층신경망(Deep Neural Network, DNN)

In [ ]:
"""
<인공신경망(Artificial Neural Network, ANN)>
 - 은닉계층(Hidden Layer)이 없는 모델
 - 입력계층 + 출력계층 or 입/출력 계층만 존재하는 모델

<심층신경망(Deep Neural Network, DNN)>
 - 은닉계층(Hidden Layer)이 있는 모델
 - 입력계층 + 은닉계층....n개 + 출력계층
"""

In [ ]:
### 1 모델 생성하기 : 변수명  model
model = keras.Sequential()
model

In [ ]:
### 2. 입력계층 생성하겨 모델에 추가하기
# - 차원을 변환하는 전처리 계층을 입력계층으로 사용
#   -> 모델에 사용되는 데이터는 2차원을 사용해야함
#   -> (축, 행, 열) => (행, 열)로 변환
#                      --> 축이 행이됨, (행*열)을 열로 통합
# - Flatten : 차원변경 전처리 계층
#           : 전처리 계층은 훈련에 사용되지 않는 계층임(데이터 전처리만 수행함)
#           : 보통 입력된 데이터의 갯수가 출력으로 사용됨
#
# - input_shape을 사용하여 입력 계층의 역할도 함께 수행함
dense1 = keras.layers.Flatten(input_shape=(28, 28))

### 모델에 계층 추가하기
model.add(dense1)
model

In [ ]:
### 은닉계층(Hidden Layer) 생성 및 모델에 추가하기
# - 입력으로 받아올 데이터 갯수 :
# - 다음계층으로 넘겨줄 데이터 갯수 : 100개(출력 갯수)
# - 훈련 중 분류에 사용할 활성화함수 activation :
hidden_layer = keras.layers.Dense(units=100, activation="relu")

### 모델에 추가하기
model.add(hidden_layer)

In [ ]:
### 출력계층 생성 및 모델에 추가하기
# - 출력 데이터 갯수 : 종속변수의 범주 갯수를 정의함
output_layer = keras.layers.Dense(units=10, activation="softmax")

model.add(output_layer)

In [ ]:
model.summary()

### 모델 생성하는 함수 정의하기

In [ ]:
"""
<함수 프로세스>
 - 함수 이름 : model_fn
 - 매개변수 : add_layer (은닉계층이 있으면 모델 중간에 추가, 없으면 추가 안함)
 - 모델 변수명 : model
 - 입력계층 모델에 추가 : 1차원 전처리 계층 사용
 - 은닉계층 추가 조건 : 여러개의 은니계층이 넘어 올 수 있음
   -- add_layer의 값이 None이 아니면, 은닉계층을 모델에 추가
   --                  None이면, 추가하지 않기
 - 출력계층 모델에 추가
 - 모델을 반환(리턴)

 -> 함수 호출 시 인자값으로 사용할 -> 테스트를 위한 은닉계층-1
    : 출력 100개, 활성화함수 relu 사용
    : 단, 여러개의 은닉계층을 인자값으로 넘길 수 있도록 처리

    : 은닉계층 2개를 사용하는 경우 추가 은닉계층-2
      -> 출력 50개, 활성화함수 relu 사용

"""

In [ ]:
def model_fn(add_layer = None) :
    ### 모델 생성하기
    model = keras.Sequential()

    ### 입력계층(차원축소 전처리 계층)  생성 및 모델에 추가하기
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    ### 은닉계층 추가여부 확인하기
    if add_layer != None :
        ### 은닉계층 여러개를 리스트 타입으로 받아서 처리할 예정
        for layer in add_layer :
            model.add(layer)

    ### 출력계층
    model.add(keras.layers.Dense(units=10, activation="softmax"))

    ### 모델 반환하기
    return model


In [ ]:
### 함수 호출하여 모델 생성하기
# - 은닉계층 없이 생성
model_temp = model_fn()
model_temp.summary()

In [ ]:
### 함수 호출하여 모델 생성하기
# - 은닉계층 추가
hidden_layer1 = keras.layers.Dense(units=100, activation="relu")

model_temp = model_fn([hidden_layer1])
model_temp.summary()

In [ ]:
### 함수 호출하여 모델 생성하기
# - 은닉계층 추가
hidden_layer1 = keras.layers.Dense(units=100, activation="relu")
hidden_layer2 = keras.layers.Dense(units=50, activation="relu")

model_temp = model_fn([hidden_layer1, hidden_layer2])
model_temp.summary()

### 모델 환경설정하기(역전파: 손실을 줄이기 위한 평가 등을 진행)

In [ ]:
### 다중분류를 진행하며, 종속변수의 값 형태는 0, 1, 2,....
# - 평가 결과는 손실율과 정확도까지 출력...
model_temp.compile(
    loss = "sparse_categorical_crossentropy",
    metrics = "accuracy"
)

### 심층신경망 훈련 시키기

In [ ]:
# 이후 모델 정의 및 fit 실행
model_temp.compile(
    loss = "sparse_categorical_crossentropy",
    metrics = "accuracy"
)
model_temp.fit(train_scaled, train_target, epochs=50)


In [ ]:
from tensorflow.keras.utils import to_categorical
train_target_oh = to_categorical(train_target)

model_temp.compile(
    loss = "categorical_crossentropy",
    metrics = "accuracy"
)
model_temp.fit(train_scaled, train_target_oh, epochs=50)


In [ ]:
import os
os.environ["TF_DETERMINISTIC_OPS"] = "0"

# 그 다음!
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 모델 정의 예시
model_temp = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(train_scaled.shape[1],)),
    layers.Dense(3, activation='softmax')
])

model_temp.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 훈련
model_temp.fit(train_scaled, train_target, epochs=50)

In [ ]:
"""
 - 훈련 횟수 50회
"""

model_temp.fit(train_scaled, train_target, epochs=50)

In [ ]:
### 훈련 결과에 대한 히스토리 정보 받아오기
with tf.device("/CPU:0") :
    history = model_temp.fit(train_scaled, train_target, epochs=50)

In [ ]:
history

In [ ]:
### 반복횟수 데이터 추출하기
print(history.epoch)

In [ ]:
### 손실율과 정확도에 대한 훈련 결과데이터 추출하기
history.history

### 훈련 중 발생한 손실율을 이용하여 손실곡선 시각화 하기

In [ ]:
### 손실곡선 : 선그래프로 그리기
# - x축은 반복횟수
# - y축은 손실율

# - 그래프는 saveFig 폴더에 png 확장자를 가지는 이미지로 저장하기

plt.title("model_epoch_loss_01")
plt.plot(history.epoch, history.history["loss"])

plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.grid()

### 이미지 저장
plt.savefig("./saveFig/model_epoch_loss_01.png")

plt.show()


### 훈련결과에 대한 정확도 곡선 시각화하기

In [ ]:
### 정확도 곡선 : 선그래프로 그리기
# - x축은 반복횟수
# - y축은 정확도
plt.title("model_epoch_accuracy_01")
plt.plot(history.epoch, history.history["accuracy"])

plt.xlabel("Epoch")
plt.ylabel("accuracy")

plt.grid()

### 이미지 저장
plt.savefig("./saveFig/model_epoch_accuracy_01.png")

plt.show()

In [ ]:
"""
 - 신규모델 생성하여 훈련하기
 - 은닉계층은 출력 100개로 한개 계층 추가
 - 반복횟수는 50회
 - 손실곡선 및 정확도 곡선 그리기..
"""
hidden_layer = keras.layers.Dense(units=100, activation="relu")

model = model_fn([hidden_layer])

model.compile(
    loss = "sparse_categorical_crossentropy",
    metrics = "accuracy"
)

with tf.device("/CPU:0") :
    history = model.fit(train_scaled, train_target, epochs=50)

In [ ]:
### 손실곡선 : 선그래프로 그리기
# - x축은 반복횟수
# - y축은 손실율

# - 그래프는 saveFig 폴더에 png 확장자를 가지는 이미지로 저장하기

plt.title("model_epoch_loss_02")
plt.plot(history.epoch, history.history["loss"])

plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.grid()

### 이미지 저장
plt.savefig("./saveFig/model_epoch_loss_02.png")

plt.show()


In [ ]:
### 정확도 곡선 : 선그래프로 그리기
# - x축은 반복횟수
# - y축은 정확도
plt.title("model_epoch_accuracy_02")
plt.plot(history.epoch, history.history["accuracy"])

plt.xlabel("Epoch")
plt.ylabel("accuracy")

plt.grid()

### 이미지 저장
plt.savefig("./saveFig/model_epoch_accuracy_02.png")

plt.show()

In [ ]:
### 손실과 정확도 곡선 : 선그래프로 그리기
# - x축은 반복횟수
# - y축은 손실율 및 정확도

# - 그래프는 saveFig 폴더에 png 확장자를 가지는 이미지로 저장하기

plt.title("model_epoch_loss_and_accuracy_03")
plt.plot(history.epoch, history.history["loss"])
plt.plot(history.epoch, history.history["accuracy"])

plt.xlabel("Epoch")
plt.ylabel("Loss and Accuracy")

plt.legend(["loss", "accuracy"])

plt.grid()

### 이미지 저장
plt.savefig("./saveFig/model_epoch_loss_and_accuracy_03.png")

plt.show()


### 훈련 및 검증 데이터를 모두 사용하여 훈련 시키기

In [ ]:
### 은닉계층
hidden_layer = keras.layers.Dense(units=100, activation="relu")

### 모델 생성 함수 호출
model = model_fn([hidden_layer])

### 모델 설정하기
model.compile(
    loss = "sparse_categorical_crossentropy",
    metrics = "accuracy"
)

### 훈련 및 검증 데이터를 이용하여 훈련 시키기
# - 훈련시 훈련에 대한 손실/정확도, 검증에 대한 손실/정확도를 모두 출력해줍니다.
with tf.device("/CPU:0") :
    history = model.fit(train_scaled, train_target, epochs=50,
                        validation_data = (val_scaled, val_target))

In [ ]:
### 훈련 및 검증 손실 곡선 시각화

plt.title("model_epoch_train_and_val_loss_04")
plt.plot(history.epoch, history.history["loss"])
plt.plot(history.epoch, history.history["val_loss"])

plt.xlabel("Epoch")
plt.ylabel("Loss")

plt.legend(["train_loss", "val_loss"])

plt.grid()

### 이미지 저장
plt.savefig("./saveFig/model_epoch_train_and_val_loss_04.png")

plt.show()

In [ ]:
### 훈련 및 검증 정확도 곡선 시각화

plt.title("model_epoch_train_and_val_accuracy_05")
plt.plot(history.epoch, history.history["accuracy"])
plt.plot(history.epoch, history.history["val_accuracy"])

plt.xlabel("Epoch")
plt.ylabel("accuracy")

plt.legend(["train_accuracy", "val_accuracy"])

plt.grid()

### 이미지 저장
plt.savefig("./saveFig/model_epoch_train_and_val_accuracy_05.png")

plt.show()